In [1]:
# importing modules
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import discretization as di
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# definning a fucntion for getting the number of DoF for a mesh
def calc_mesh_dof( ele_type, coords):
    _, ndof, _ = di.element_info(ele_type)
    n_nodes = coords.shape[0]
    mesh_ndof = ndof * n_nodes
    return mesh_ndof

def solve_hyperelastic_rect_mesh(ele_type, L_x, L_y, N_x, N_y, q, nr_num_steps):
    """Solves a 2D hyperelastic problem on a rectangular mesh."""
    
    #--- Mesh Parameters ---
    origin_x, origin_y = 0.0, 0.0
    num_gauss_pts = 1  # Currently unused

    #--- Get Element Info ---
    _, ndof, _ = di.element_info(ele_type)

    #--- Generate Mesh ---
    coords, connect = pre.generate_rect_mesh_2d(
        ele_type,
        origin_x, origin_y,
        origin_x + L_x, origin_y + L_y,
        N_x, N_y
    )

    #--- Define Boundaries ---
    boundary_nodes, boundary_edges = pre.identify_rect_boundaries(
        coords, connect, ele_type,
        origin_x, origin_x + L_x,
        origin_y, origin_y + L_y
    )

    # Fixed boundary on the left edge: u_x = u_y = 0
    fixed_nodes = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.0, 0.0)

    # Apply uniform load on the right edge in x-direction
    dload_info = pre.assign_uniform_load_rect(boundary_edges, "right", 0.0, q)

    #--- Material Properties ---
    mu = 10.0
    kappa = 1000.0
    material_props = np.array([mu, kappa])

    # Optional: derived properties for reference
    nu = (3 * kappa - 2 * mu) / (6 * kappa + 2 * mu)
    E = 2 * mu * (1 + nu)

    #--- Solve Nonlinear Problem ---
    displacements_all, nr_info_all = hyperelastic_solver(
        material_props,
        ele_type,
        coords.T,
        connect.T,
        fixed_nodes,
        dload_info,
        nr_print=True,
        nr_num_steps=nr_num_steps,
        nr_tol=1e-9,
        nr_maxit=30
    )

    return coords, connect, displacements_all


In [3]:
#--- Geometry Parameters ---
ratio = 5
L_y = 1                          # Width
L_x = ratio * L_y               # Length

#--- Mesh Sizes: (N_x, N_y) ---
mesh_size = ratio * 10, 10


#--- Element and Solver Setup ---
ele_type = 'D2_nn4_quad'        # Type of element for the mesh
q = -1                       # Applied traction in x-direction
nr_num_steps = 2               # Number of Newton-Raphson steps

#--- Run Simulation and Collect Results ---
meshsize2result_D2nn4quad_dict = {'ele_type': ele_type}

N_x, N_y = mesh_size
coords, connect, displacements_all = solve_hyperelastic_rect_mesh(
    ele_type, L_x, L_y, N_x, N_y, q, nr_num_steps
)
mesh_ndof = calc_mesh_dof(ele_type, coords)
meshsize2result_D2nn4quad_dict[mesh_size] = (
    coords,
    connect,
    displacements_all[-1],  # Final displacement field
    mesh_ndof
)


Step 0, load factor = 0.500
Iteration 1, Correction=1.000000e+00, Residual=1.373533e-04, tolerance=1.000000e-09
Iteration 2, Correction=4.410400e-01, Residual=2.299147e+00, tolerance=1.000000e-09
Iteration 3, Correction=0.000000e+00, Residual=nan, tolerance=1.000000e-09
Step 1, load factor = 1.000
Iteration 1, Correction=0.000000e+00, Residual=nan, tolerance=1.000000e-09


/home/sara/PhD/ME700/HW3_new/finite-element-analysis/src/finiteelementanalysis/solver.py:133: MatrixRankWarning: Matrix is exactly singular
  d_displacement = spla.spsolve(K_sparse, R)
/home/sara/anaconda3/envs/finite-element-analysis-env/lib/python3.12/site-packages/numpy/linalg/_linalg.py:2432: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


**Reason for failure**  
Reason for Failure
In this example, the applied force is significantly larger than in Parts A and B, and the parameters are different as well. Because of this, the solver struggles to converge:

Large Force
When the force is much larger, the system can experience bigger displacements, stresses, or other responses. If the numerical solver tries to handle these large changes in one big step (or with very few steps), the predicted solution might jump too far from the true solution. This large “jump” can cause the iterative solver to diverge.

Poor Initial Condition
Most iterative solvers (e.g., Newton-Raphson) start from an initial guess. If that guess is too far from the true solution—especially when forces are large—the solver’s iterative updates can fail to converge. Essentially, each iteration corrects the guess based on local information (e.g., local gradients, residuals). With a big force and a bad initial guess, the solver updates can overshoot.


Breaking the simulation into more, smaller steps (by increasing nr_num_steps) forces the solver to approach the solution more gradually. Rather than applying the entire load or change at once, you apply it incrementally. Each step starts from the converged solution of the previous step, which becomes a much better initial guess for the next increment. This approach often stabilizes the solver and prevents divergence.

